<a href="https://colab.research.google.com/github/pallateja96/Alage-detectection-CNES/blob/main/sentinel_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=tapsVF03liPqwQd5RY6qMO5JTS7oubSOygPdKT1MUvQ&tc=P0x3_C-z9Y1ogXKr6JD8K9yiyG5aMp5KQ5H1gETWDy4&cc=5euY7DxJJZ8lF9Fc4EqC7kLLmir4MQrK_mmPNCW6bIY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh5X5hARkAa6twp4bT6qgZwMDkqzTVNuqjbIcVAl33eYI5nf4uBhaBw

Successfully saved authorization token.


In [18]:
import folium
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Define the region of interest as a polygon
aoi = ee.Geometry.Polygon(
        [[[-97.53036215783045, 24.968390607847912],
          [-97.53036215783045, 16.174856991797686],
          [-84.91805747033045, 16.174856991797686],
          [-84.91805747033045, 24.968390607847912]]])

# Define the Sentinel 3 image collection
collection = ee.ImageCollection('COPERNICUS/S3/OLCI')

# Filter the collection by date and region of interest
image = collection.filterDate('2022-07-07', '2022-07-12').filterBounds(aoi)

# Select the bands you want to use for the red, green, and blue channels
bands = ['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance']

# Create a truecolor composite of the image
rgb = image.select(bands).mean().visualize(min=0, max=6000, gamma=1.5)

# Convert the image to a 8-bit RGB image
rgb_8bit = rgb.multiply(256).toByte()

# Get the URL of the image
url = rgb_8bit.getThumbURL({'dimensions': 512, 'region': aoi})

# Create a folium map
m = folium.Map(location=[24.968390607847912, -97.53036215783045])

# Add the image to the map
folium.raster_layers.ImageOverlay(url, bounds=[[16.174856991797686, -97.53036215783045], [24.968390607847912, -84.91805747033045]], opacity=0.8).add_to(m)

# Display the map
m






In [17]:
# Define the export parameters
scale = 10
crs = 'EPSG:4326'
region = aoi.bounds().getInfo()['coordinates']

# Export the image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=rgb,
    description='Sentinel3_OLCI_TrueColor',
    folder='Sentinel3',
    fileNamePrefix='Sentinel3_OLCI_TrueColor',
    scale=scale,
    crs=crs,
    region=region
)

# Start the export task
task.start()

# Print the task status
print(task.status())






{'state': 'READY', 'description': 'Sentinel3_OLCI_TrueColor', 'creation_timestamp_ms': 1674986436167, 'update_timestamp_ms': 1674986436167, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UE665ED32OMWQOSJGYXFVZBM', 'name': 'projects/earthengine-legacy/operations/UE665ED32OMWQOSJGYXFVZBM'}
